In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager

# Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def scrape_song_links(driver, max_songs=100):
    url = "https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H/discography/album"
    driver.get(url)
    time.sleep(5)  # 先給頁面一點時間載入

    song_links = set()
    body = driver.find_element(By.TAG_NAME, "body")

    last_count = 0          # 上一次抓到幾首
    same_count_times = 0    # 有幾次滾動後數量都沒變

    while len(song_links) < max_songs and same_count_times < 5:
        # 1. 先抓目前畫面上所有 /track/ 連結
        elements = driver.find_elements(By.XPATH, "//a[contains(@href, '/track/')]")
        for elem in elements:
            href = elem.get_attribute("href")
            if href and "/track/" in href:
                song_links.add(href)

        print("Current songs:", len(song_links))

        # 2. 往下滾動一頁
        body.send_keys(Keys.END)
        time.sleep(2)  # 等新內容載入

        # 3. 檢查有沒有新增歌曲
        if len(song_links) == last_count:
            same_count_times += 1
        else:
            same_count_times = 0
            last_count = len(song_links)

    print(f"Total songs found: {len(song_links)}")
    # 只要最多 max_songs 首
    return list(song_links)[:max_songs]


In [5]:
links = scrape_song_links(driver, max_songs=100)
for link in links:
    print(link)


Current songs: 46
Current songs: 69
Current songs: 93
Current songs: 132
Total songs found: 132
https://open.spotify.com/track/390Zbl3vRpmvjyliv3cp96
https://open.spotify.com/track/2GhqFR4Yw3GLpBNpnelfLi
https://open.spotify.com/track/1gqrHlva1UviyDTl1Bobsj
https://open.spotify.com/track/3Tb4oHMR617zvLANuiV1ok
https://open.spotify.com/track/5I6KaV4xAc6ZYyOgEtp6dc
https://open.spotify.com/track/6cJoyuvyTeNrxhEhlNBmUv
https://open.spotify.com/track/0xosb9d4wbXTYJ2FIbUm0h
https://open.spotify.com/track/36su1AYFo1yBgtT8kVV2nN
https://open.spotify.com/track/0U10zFw4GlBacOy9VDGfGL
https://open.spotify.com/track/25A1RmT5eh7C99KXfESRj5
https://open.spotify.com/track/0GLfodYacy3BJE7AI3A8en
https://open.spotify.com/track/03cL2gI39EAUUEwpi3mIfr
https://open.spotify.com/track/4oHkmDMdEHbd0f1xWqyFC7
https://open.spotify.com/track/7lUhtLCdqomO7jsZFkVcBW
https://open.spotify.com/track/49tG0W4vxwEQtpr0FfkzTQ
https://open.spotify.com/track/7tcxXaD8pklUOIUIrwVUhx
https://open.spotify.com/track/2q03jmzxg

In [ ]:
link.to_csv("song_links.csv", index=False)